# which words are expressible on a graph?
given the following graph of allowed pairs of characters:

![connected graph of allowed pairs of characters](../img/puzzler.png)

write a program that can read a dictionary file (e.g. `/usr/share/dict/words`) and output only the words that are expressible on this graph. 
characters do not connect to themselves, while the letters and connections can be reused. 
find and report the longest dictionary word expressed this way.

## graph details
the graph above has the following 13 nodes and 72 (directed) vertices:

```
nodes=[
    'b', 'r', 'c', 'h', 's', 
    'y', 'l', 'e', 'g', 'w', 
    'n', 't', 'o']
vertices = [
    "br", "bl", "bo", "bt", "by", 
    "rc", "re", "ro", "rl", "rb",
    "ch", "cg", "co", "ce", "cr",
    "hs", "hw", "ho", "hg", "hc",
    "sy", "sn", "so", "sw", "sh",
    "yb", "yt", "yo", "yn", "ys",
    "tb", "tl", "to", "tn", "ty",
    "lr", "le", "lo", "lt", "lb",
    "er", "ec", "eg", "eo", "el",
    "ge", "gc", "gh", "gw", "go",
    "wo", "wg", "wh", "ws", "wn",
    "nt", "no", "nw", "ns", "ny",
    "ob", "or", "oc", "oh", "os",
    "oy", "ol", "oe", "og", "ow",
    "on", "ot"
]
```


In [6]:
# the graph above can be expressed as a dict, wherein
# given a letter key, the corresponding value lists 
# the allowed letters which may follow it:
next_letter_dict = {
    'b': ['r','l','o','t','y'],
    'r': ['c','e','o','l','b'],
    'c': ['h','g','o','e','r'],
    'h': ['s','w','o','g','c'],
    's': ['y','n','o','w','h'],
    'y': ['b','t','o','n','s'],
    't': ['b','l','o','n','y'],
    'l': ['r','e','o','t','b'],
    'e': ['r','c','g','o','l'],
    'g': ['e','c','h','w','o'],
    'w': ['o','g','h','s','n'],
    'n': ['t','o','w','s','y'],
    'o': ['b','r','c','h','s', 
          'y','l','e','g','w', 
          'n','t'],
}

# nodes
allowed_tokens = set(next_letter_dict.keys())

In [7]:
# single word tests:
# word = 'wrongness'
# word = 'wongness'
word = 'wonoss'

tokens = set(word)
word_length = len(word)
print('input is', word)
print('which contains', tokens)
if tokens.issubset(allowed_tokens):
    print('all letters allowed, searching for path')
    if len(word) > 1:
        print('testing one letter pair at a time')
        allowed = True
        first_letter = word[0]
        for second_letter in word[1:]:
            if second_letter not in next_letter_dict[first_letter]:
                allowed = False
                print('the pair', first_letter, '->', second_letter, 'is not allowed')
                break
            first_letter = second_letter
    else:
        allowed = True
        print('word', word, 'is allowed')
else:
    allowed = False
    print('word', word, 'contains characters not included in graph')
    print('missing letters:', tokens - allowed_tokens)
if allowed:
    print('>>>>>>the word', word, 'is allowed')
else:
    print('>>>>>>the word', word, 'is impossible')

input is wonoss
which contains {'s', 'n', 'w', 'o'}
all letters allowed, searching for path
testing one letter pair at a time
the pair s -> s is not allowed
>>>>>>the word wonoss is impossible


In [34]:
# function wrap up
def check_if_expressible(word, next_letter_dict=next_letter_dict, allowed_tokens=allowed_tokens):
    """
    given a string and a graph of allowed token pairings, this function checks whether the string can 
    be formed by traversing the graph of allowed pairs. 
    input: 
        word, a character string.
        next_letter_dict, a dict whose keys are all the allowed tokens, the values are each key's (token's) allowed pairings.
        allowed_tokens, a set of tokens, equal to the keys of the next letter dict.
    output:
        allowed, boolean. true if word can be generated by traversing the graph.
    """
    if not set(word).issubset(allowed_tokens):
        # input word contains tokens not found in graph of allowed pairs.
        allowed = False
#         print('word', word, 'contains characters not included in graph')
#         print('missing letters:', set(word) - allowed_tokens)
    else:
        if len(word) > 1:
            # need to check pairs of tokens
            allowed =True
            first_letter = word[0]
            for second_letter in word[1:]:
#                 print(word, 'checking the pairs', first_letter, second_letter)
                if second_letter not in next_letter_dict[first_letter]:
                    allowed = False
#                     print('the pair', first_letter, '->', second_letter, 'is not allowed')
                    break
                first_letter = second_letter
        else:
            allowed = True
#     if allowed:
#         print('>>>>>>the word', word, 'is allowed')
#     else:
#         print('>>>>>>the word', word, 'is impossible')
    return allowed


# def read_word_list(filepath):
#     word_list = []
#     with open(filepath, 'r') as file:
#         for line in file:
#             word_list.append(file.readline().strip())
#     return word_list

def read_word_list(filepath):
    with open(filepath, 'r') as file:
        word_list= file.readlines()
    return [word.strip() for word in word_list]



def scan_words(word_list):
    return [
        word 
        for word in word_list
        if check_if_expressible(word)
    ]
            
    

In [35]:
check_if_expressible('wongness')

False

In [42]:
%time
import random
words = read_word_list('../data/words.txt')
print('word list `words` contains', len(words), 'words, e.g.', random.sample(words, 5))
allowed_words = scan_words(words)
print('of which', len(allowed_words), 'can be formed using graph')
long_allowed = [word for word in allowed_words if len(word)>6]
print('of which', len(long_allowed), 'are long')
print('for example:', random.sample(long_allowed, 4))
max_length = 5
for word in long_allowed:
    word_length = len(word)
    if word_length > max_length:
        longest_word = word
        max_length = word_length
print('and the longest word found was', longest_word, '(',  max_length, 'letters )')

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs
word list `words` contains 235886 words, e.g. ['otoneuralgia', 'servitress', 'unfidgeting', 'ranged', 'Nematognathi']
of which 404 can be formed using graph
of which 36 are long
for example: ['geologer', 'gonyocele', 'torotoro', 'reconsole']
and the longest word found was gonyocele ( 9 letters )


In [41]:
# allowed_words
long_allowed

['gonyocele', 'horologer', 'reconsole', 'snowshoer']